# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique:
* Model:
* Evaluation approach:
* Fine-tuning dataset:

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [ ]:
!pip install -U accelerate
!pip install -U transformers
!pip install peft

  Using cached accelerate-0.28.0-py3-none-any.whl (290 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q "datasets==2.15.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

# The sms_spam dataset only has a train split, so we use the train_test_split method to split it into train and test
dataset = load_dataset("sms_spam", split="train").train_test_split(
test_size=0.2, shuffle=True, seed=23,
)

splits = ["train", "test"]

# View the dataset characteristics
dataset["train"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

Dataset({
    features: ['sms', 'label'],
    num_rows: 4459
})

In [ ]:
from transformers import AutoTokenizer

def get_tokenizer(model_name):
    return AutoTokenizer.from_pretrained(model_name,
                                          bos_token="[BOS]",
                                          eos_token="[EOS]",
                                          unk_token="[UNK]",
                                          pad_token="[PAD]",
                                  mask_token="[MASK]",
                                  )

In [ ]:
model_name = "distilbert-base-uncased"


In [ ]:
# preprocess


tokenizer = get_tokenizer(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["sms"], padding="max_length",\
                                          truncation=True,\
                                        max_length = tokenizer.model_max_length), batched=True
    )

# Inspect the available columns in the dataset
tokenized_dataset["train"]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/4459 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 4459
})

In [ ]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": accuracy_score(labels, predictions)}


In [ ]:
model_name

'distilbert-base-uncased'

In [ ]:
from transformers import AutoModelForSequenceClassification


model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                             num_labels=2,
                                             id2label={0: "not spam", 1: "spam"},
                                             label2id={"not spam": 0, "spam": 1},)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

Embedding(30524, 768)

In [ ]:
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
for param in model.parameters():
    param.requires_grad = True

In [ ]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30524, 768)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin

In [ ]:
def get_trainer(model):
  return Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=3,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=False,
    ),
    train_dataset=tokenized_dataset["train"].rename_column('label', 'labels'),
    eval_dataset=tokenized_dataset["test"].rename_column('label', 'labels'),
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [ ]:
foundation_model_trainer = get_trainer(model)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
foundation_model_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.555000,0.443882,0.870852
2,0.532600,0.498786,0.870852
3,0.484600,0.488339,0.870852


TrainOutput(global_step=3345, training_loss=0.5352232062050402, metrics={'train_runtime': 776.5096, 'train_samples_per_second': 17.227, 'train_steps_per_second': 4.308, 'total_flos': 1772016391821312.0, 'train_loss': 0.5352232062050402, 'epoch': 3.0})

In [ ]:
predict,label_ids, foundation_model_metrics = foundation_model_trainer.predict(tokenized_dataset['test'])

In [ ]:
foundation_model_metrics

{'test_loss': 0.4883394241333008,
 'test_accuracy': 0.8708520179372198,
 'test_runtime': 20.988,
 'test_samples_per_second': 53.126,
 'test_steps_per_second': 13.293}

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                             num_labels=2,
                                             id2label={0: "not spam", 1: "spam"},
                                             label2id={"not spam": 0, "spam": 1},)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(30524, 768)

In [ ]:
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
from peft import LoraConfig, TaskType
config = LoraConfig(inference_mode=False, r=18, lora_alpha=32, lora_dropout=0.1,modules_to_save=["classifier"], bias='lora_only',
                    target_modules=['lin1'],     task_type="SEQ_CLS"

                    )

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
from peft import get_peft_model
lora_model = get_peft_model(model, config)

In [ ]:
lora_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30524, 768)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(in_features=768, out_features=768, bias=True)
                (k_lin): Linear(in_features=768, out_features=768, bias=True)
                (v_lin): Linear(in_features=768, out_features=768, bias=True)
                (out_lin): Linear(in_features=768, out_features=768, bias=True)
              )
              (sa_layer_norm

In [ ]:
lora_trainer = get_trainer(lora_model)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
lora_model.print_trainable_parameters()


trainable params: 1,025,282 || all params: 67,963,396 || trainable%: 1.5085797066409101


In [ ]:
lora_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.246000,0.522730,0.870852
2,0.286500,0.255706,0.849327
3,0.244700,0.258779,0.870852


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=3345, training_loss=0.2993639137712829, metrics={'train_runtime': 559.6014, 'train_samples_per_second': 23.905, 'train_steps_per_second': 5.977, 'total_flos': 1813392030707712.0, 'train_loss': 0.2993639137712829, 'epoch': 3.0})

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [ ]:
lora_model.save_pretrained("/content/drive/MyDrive/udacity_gen_ai_model/distillbert_lora", save_embedding_layers=True)


In [ ]:
from peft import AutoPeftModelForSequenceClassification
lora_model = AutoPeftModelForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/udacity_gen_ai_model/distillbert_lora",
    ignore_mismatched_sizes=True,
    num_labels = len(dataset['train'].features['label'].names))



In [ ]:
predict,label_ids,lora_model_metrics = lora_trainer.predict(tokenized_dataset['test'])


In [ ]:
lora_model_metrics

{'test_loss': 0.258778840303421,
 'test_accuracy': 0.8708520179372198,
 'test_runtime': 20.7413,
 'test_samples_per_second': 53.757,
 'test_steps_per_second': 13.451}

# Conclusion

Comparing the traning results of both Foundational model and model trained with LORA PEFT. Here are the observations
- Test loss decreased to almost half while training with LORA compared to foundational model.
- Test accuracy for both the pretrained model and LORA model for 3 epochs.
- Test runtime for both are same.
- Test sample per second and test steps per second are almost same.